# my own simple linear regression
this is my attempt at creating a linear regression model from scratch  
The model should have the same user interface as sklearn  
model = LinearRegression()  
model.fit(X, Y)  

The only external Resource for this Implementation is the following Formula for regularized Gradient descent, taken from the Original Machine Learning Course from Andrew NG

![formula](./regularized_linear_regression.png)

for now, all we need is numpy

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
class linear_regression_model():
    def __init__(self):
        self.theta = None
        self.J_histories = []
        self.theta = None
    
    def predict(self, X):
        X = np.hstack((np.ones((len(X),1)), X))
        predictions = X @ self.theta
        return predictions
        
    def fit(self, X:np.ndarray, y:np.ndarray, alpha = 0.1, num_iters = 100_000, y_lambda = 0):
        # add ones as an extra feature (at the beginning) in the input, to use this for the offset term
        X = np.hstack((np.ones((len(X),1)), X))
        
        if self.theta is None:
            self.theta = np.random.rand(X.shape[1], 1) # initialize 
        J_history = self.gradient_descent_regularised(X, y, self.theta, alpha, num_iters, y_lambda)
        self.J_histories.append(J_history)
    
    def compute_cost_add_offset(self, X, y):
        # add ones as an extra feature (at the beginning) in the input, to use this for the offset term
        X = np.hstack((np.ones((len(X),1)), X))
        if self.theta is None:
            self.theta = np.random.rand(X.shape[1], 1) # initialize 
        m = len(y) # number of training examples
        J = 1/(2*m) * sum((X @ self.theta - y)**2)
        return J
        
    def compute_cost(self, X, y):
        m = len(y) # number of training examples
        J = 1/(2*m) * sum((X @ self.theta - y)**2)
        return J
    
    def gradient_descent_regularised(self, X:np.ndarray, y:np.ndarray, theta:np.ndarray, alpha, num_iters, y_lambda):
        m = len(y)
        J_history = np.empty(num_iters)

        for i in range(num_iters):
            # % Instructions: Perform a single gradient step on the parameter vector
            # %               theta. 
            theta = self.theta
            J = X @ theta - y
            # Save the cost J before every iteration    
            J_history[i] = ( 1/(2*m) * sum((J)**2) )
            
            self.theta = theta * ( 1 - alpha*y_lambda/m) - (alpha/m) * np.transpose(np.transpose( J ) @ X)
        
        return theta, J_history

## Try out my model

In [3]:
model = linear_regression_model()

In [4]:
n_examples = 4
n_features = 2
X = np.random.rand(n_examples, n_features)
y = np.random.rand(n_examples, 1)

In [5]:
model.compute_cost_add_offset(X, y)

array([0.06889351])

In [6]:
model.fit(X, y)

In [8]:
model.compute_cost_add_offset(X, y)

array([0.01075474])

In [9]:
p0 = model.predict(X)
p0

array([[0.26410102],
       [0.67226816],
       [0.18369382],
       [0.36611077]])

# Compare my model to sklearn and statsmodels

In [10]:
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

In [11]:
sklearn_model = LinearRegression()
sklearn_model.fit(X,y)

LinearRegression()

In [12]:
p1 = sklearn_model.predict(X)
p1

array([[0.26410102],
       [0.67226816],
       [0.18369382],
       [0.36611077]])

## how much percent am I off?

In [13]:
(p1 - p0)*100

array([[-2.05391260e-13],
       [ 2.55351296e-13],
       [-1.97064587e-13],
       [ 1.11022302e-13]])

## experiment to find out how my algorithm compares to the reference

In [14]:
my_diffs = []
sk_diffs= []
for i in range(10):
    n_examples = 100
    n_features = 10
    X = np.random.rand(n_examples, n_features)
    y = np.random.rand(n_examples, 1)
    
    # my model
    my_model = linear_regression_model()
    my_model.fit(X, y)
    my_predictions = my_model.predict(X)
    
    # sklearn
    sk_model = LinearRegression()
    sk_model.fit(X,y)
    sk_predictions = sk_model.predict(X)
    
    # statsmodels
    X1 = sm.add_constant(X)
    sm_model = sm.OLS(y, X1).fit()
    sm_predictions = np.vstack(sm_model.predict(X1))
    
    # we pretend that statsmodels is the ground truth and then we see, 
    # how far my model is of in comparison to the sklearn implementation
    
    my_diff = np.sqrt(sum((my_predictions - sm_predictions)**2))
    my_diffs.append(my_diff)
    sk_diff = np.sqrt(sum((sk_predictions - sm_predictions)**2))
    sk_diffs.append(sk_diff)
    
np.hstack([my_diffs, sk_diffs])

array([[3.86422609e-14, 1.23419654e-14],
       [1.74749069e-14, 2.17133058e-15],
       [3.80556878e-14, 3.02803026e-15],
       [3.60846395e-14, 1.37307929e-14],
       [2.41661024e-14, 2.57753524e-15],
       [4.46444616e-14, 2.91012412e-14],
       [2.71076503e-14, 1.28256590e-14],
       [6.66431369e-14, 5.62839413e-14],
       [3.40812701e-14, 5.00463123e-15],
       [2.61537654e-14, 5.22159028e-15]])

### interpretation
The learning rate and the number of iterations in my algorithm are adjusted, so that the accuracy is similar to the sklearn implementation  
my implementation takes significantly longer, because I use the vanilla gradient descent implementation.  
And I did not implement any Improvements like a varying learning rate and an automatic detection of "learning finished".

## investigate the time performance

In [15]:
n_examples = 100
n_features = 10
X = np.random.rand(n_examples, n_features)
y = np.random.rand(n_examples, 1)

# my model
my_model = linear_regression_model()

### model learning

In [16]:
###########    My Model
my_model = linear_regression_model()

In [17]:
%timeit my_model.fit(X, y)

5.75 s ± 95 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
###########    Scikit Learn
sk_model = LinearRegression()

In [19]:
%timeit sk_model.fit(X,y)

560 µs ± 2.23 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [20]:
# statsmodels
X1 = sm.add_constant(X)
sm_model = sm.OLS(y, X1).fit()

In [21]:
%timeit sm.OLS(y, X1).fit()

250 µs ± 2.56 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### predictions

In [22]:
%timeit my_model.predict(X)

9.12 µs ± 598 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [23]:
%timeit sk_model.predict(X)

42.6 µs ± 381 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
%timeit sm_model.predict(X1)

6.43 µs ± 117 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
